In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import json

# DB 접속 정보 로드
with open('db-config.json') as f:
    config = json.load(f)

user = config['user']
password = config['password']
host = config['host']
port = config['port']
database = config['database']

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4")

# 원본 데이터 불러오기
air_pollution = pd.read_sql("SELECT * FROM air_pollution", con=engine)
air_quality = pd.read_sql("SELECT * FROM air_quality", con=engine)
weather = pd.read_sql("SELECT * FROM weather", con=engine)
uv_ray_day = pd.read_sql("SELECT * FROM uv_ray_day", con=engine)

In [2]:
merged = pd.read_csv("C:/Users/user/ai-proj2-class4/10.data-collecting/자외선_데이터셋.csv")

In [ ]:
#소수점 자리수 계산 함수 정의
def get_decimal_places(df):
    result = {}
    for col in df.select_dtypes(include='float'):
        # 소수점 자리수 길이 계산 (mode를 사용해 대표값)
        decimal_lengths = df[col].dropna().apply(lambda x: len(str(x).split('.')[-1]))
        if not decimal_lengths.empty:
            result[col] = decimal_lengths.mode()[0]
    return result

In [ ]:
#원본 데이터 기준 소수점 자리수 파악
dp_air = get_decimal_places(air_pollution)
dp_quality = get_decimal_places(air_quality)
dp_weather = get_decimal_places(weather)
dp_uv = get_decimal_places(uv_ray_day)

In [5]:
# 통합된 자리수 맵
decimal_places_map = {}
for d in [dp_air, dp_quality, dp_weather, dp_uv]:
    for key, val in d.items():
        decimal_places_map[key] = val

In [6]:
#병합 데이터에 소수점 자리수 적용
for col, dp in decimal_places_map.items():
    if col in merged.columns:
        merged[col] = merged[col].round(dp)

In [7]:
merged.to_csv("자외선_데이터셋_소수점정리.csv", index=False)

In [8]:
print("최종 병합 데이터 크기:", merged.shape)

최종 병합 데이터 크기: (2027707, 14)


In [9]:
# 결측치가 없는 경우 바로 int로 변환
merged['pm10'] = merged['pm10'].astype('int')
merged['pm25'] = merged['pm25'].astype('int')

In [10]:
# datetime이 문자열일 수도 있으니 안전하게 변환
merged['datetime'] = pd.to_datetime(merged['datetime'])

# datetime 기준 정렬
merged = merged.sort_values(by='datetime').reset_index(drop=True)

In [11]:
merged.to_csv("자외선_데이터셋_최종.csv", index=False)

In [12]:
print("최종 병합 데이터 크기:", merged.shape)

최종 병합 데이터 크기: (2027707, 14)


In [13]:
from sqlalchemy import create_engine
import json

# 1) DB 접속 정보
with open('db-config.json') as f:
    config = json.load(f)

user = config['user']
password = config['password']
host = config['host']
port = config['port']
database = config['database']

# 2) SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4")

# 3) DataFrame을 테이블에 INSERT
merged.to_sql(
    name='uv_dataset',     # 업로드할 테이블명
    con=engine,
    if_exists='append',    # 기존 테이블에 이어서 넣음 (덮어쓰기: 'replace')
    index=False            # 인덱스는 안 넣음
)

print("최종 데이터 DB에 업로드 완료")

최종 데이터 DB에 업로드 완료


In [14]:
decimal_places_map = {
    'no2': 4,
    'co': 2,
    'so2': 4,
    'o3': 4,
    'pm10': 0,
    'pm25': 0,
    'temperature': 1,
    'wind_direction': 1,
    'wind_speed': 1,
    'precipitation': 0,
    'humidity': 0,
    'uv_max': 1
}

# 병합 데이터에 적용
for col, dp in decimal_places_map.items():
    if col in merged.columns:
        merged[col] = merged[col].round(dp)

# datetime 정렬
merged['datetime'] = pd.to_datetime(merged['datetime'])
merged = merged.sort_values(by='datetime').reset_index(drop=True)

In [15]:
merged.to_csv("자외선_데이터셋_최종.csv", index=False)

In [16]:
print("최종 병합 데이터 크기:", merged.shape)

최종 병합 데이터 크기: (2027707, 14)


In [17]:
merged.to_sql(
    name='uv_dataset',
    con=engine,
    if_exists='replace',  # 기존 테이블을 완전히 삭제하고 새로 생성
    index=False
)

2027707